<a href="https://colab.research.google.com/github/ismoil27/jaydariGPT/blob/main/jaydari_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install transformers torch bitsandbytes

In [22]:
from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForCausalLM
import torch

In [23]:
model_id = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'
tokenizer = AutoTokenizer.from_pretrained(model_id)

# print('Vocab size:', tokenizer.vocab_size)
# print('Special tokens:', tokenizer.special_tokens_map)

# quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type='nf4'
)

bnb_config

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map='auto', # GPU, CPU
    # dtype=torch.bfloat16
)

In [24]:
# Before Fine-tuning
prompt = "Explain what a tokenizer is?"
# prompt = "A tokenizer is a tool in natural language processing that"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device) # GPU, CPU

with torch.no_grad():
    output_ids = model.generate(
        **inputs,
        max_new_tokens=80,
        do_sample=True,
        temperature=0.7
    )

print(tokenizer.decode(output_ids[0], skip_special_tokens=True))

# print(model)
first_block = model.model.layers[0]
print('first_block:', first_block)
print('=======')
print(first_block.self_attn)
print('=======')
print(model.config)



Explain what a tokenizer is? What is the significance of tokenization in Natural Language Processing (NLP) and why is it used in Text Analytics?
first_block: LlamaDecoderLayer(
  (self_attn): LlamaAttention(
    (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
    (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
    (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
    (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
  )
  (mlp): LlamaMLP(
    (gate_proj): Linear4bit(in_features=2048, out_features=5632, bias=False)
    (up_proj): Linear4bit(in_features=2048, out_features=5632, bias=False)
    (down_proj): Linear4bit(in_features=5632, out_features=2048, bias=False)
    (act_fn): SiLUActivation()
  )
  (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
  (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
)
LlamaAttention(
  (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
  (k

In [25]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

total_params = count_parameters(model)
print(f"Total parameters (including frozen 4-bit): {total_params:,}")


Total parameters (including frozen 4-bit): 615,606,272
